In [3]:
import googlemaps
import folium
from haversine import haversine
from sklearn import preprocessing, cluster
from sklearn.metrics.pairwise import haversine_distances

from sklearn.cluster import DBSCAN
from geopy.distance import great_circle
from shapely.geometry import MultiPoint

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings(action="ignore") #경고 표시 제거는 일시적으로만 하자... 중요한 내용일수도

import matplotlib.font_manager as fm #한글 안깨지게
# font_name=fm.FontProperties(fname="C:/Windows/Fonts/malgun.ttf").get_name()
# plt.rc("font", family=font_name)

import matplotlib as mpl # - 표시 정상적으로 되도록
mpl.rcParams["axes.unicode_minus"]=False

import urllib.request as req
from urllib.request import urlopen
from urllib.error import HTTPError, URLError
import urllib.parse
from bs4 import BeautifulSoup
import requests

from selenium import webdriver
import time

In [3]:
df = pd.read_excel("data/리뷰_6개구 수정.xlsx", index_col=0)


,이름,평점,주소,주력 메뉴,가격대,망고플레이트 주소,리뷰내용
3869.0,청우참치,\n4.5\n,서울특별시 강남구 강남대로 632,\n회 / 스시\n,4만원 이상,https://www.mangoplate.com//restaurants/YyRGx0...,\n 가로수길에 있는 오랜 참치맛집 e-청우참치위치를 옮기다!!원래...
3865.0,청우참치,\n4.5\n,서울특별시 강남구 강남대로 632,\n회 / 스시\n,4만원 이상,https://www.mangoplate.com//restaurants/YyRGx0...,"\n 가성비 너무 좋고, 정말 양질의 참치를 양껏 즐길 수 있어서 ..."
3867.0,청우참치,\n4.5\n,서울특별시 강남구 강남대로 632,\n회 / 스시\n,4만원 이상,https://www.mangoplate.com//restaurants/YyRGx0...,\n 가성비 좋은 한국식 참치횟집. 선도가 꽤 좋다. 카운터석도 있...
3868.0,청우참치,\n4.5\n,서울특별시 강남구 강남대로 632,\n회 / 스시\n,4만원 이상,https://www.mangoplate.com//restaurants/YyRGx0...,\n 그저 박수만. 참치땡길때 여기만한 곳이 없다.\n
3863.0,청우참치,\n4.5\n,서울특별시 강남구 강남대로 632,\n회 / 스시\n,4만원 이상,https://www.mangoplate.com//restaurants/YyRGx0...,\n 맛있다.. 느끼해서 많이 못먹는게 함정.\n
...,...,...,...,...,...,...,...
10389.0,프로젝트얼스,\n 4.5\n ...,서울특별시 성북구 성북로15길 21,\n카페 / 디저트\n,만원 미만,https://www.mangoplate.com//restaurants/2r63mF...,\n 성북동에서 까페를 찾다가 가게되었어요. 구조가 뭔가 특이하고 ...
10390.0,프로젝트얼스,\n 4.5\n ...,서울특별시 성북구 성북로15길 21,\n카페 / 디저트\n,만원 미만,https://www.mangoplate.com//restaurants/2r63mF...,\n * 한줄평 : 아지트 삼고싶은 성북동 골목 카페성북동 언덕 아...
10391.0,프로젝트얼스,\n 4.5\n ...,서울특별시 성북구 성북로15길 21,\n카페 / 디저트\n,만원 미만,https://www.mangoplate.com//restaurants/2r63mF...,\n 조용한 성북동에있는 가격도 저렴하고 부니기 좋은 카페 ㅎㅎ 사...
10392.0,프로젝트얼스,\n 4.5\n ...,서울특별시 성북구 성북로15길 21,\n카페 / 디저트\n,만원 미만,https://www.mangoplate.com//restaurants/2r63mF...,\n 리뷰 썼다가 사람 많아질까 두렵지만... 그렇지만 또 장사가 ...


In [9]:
df=df.reset_index()
df=df.drop(["index"], axis=1)

,level_0,이름,평점,주소,주력 메뉴,가격대,망고플레이트 주소,리뷰내용
0,0,청우참치,\n4.5\n,서울특별시 강남구 강남대로 632,\n회 / 스시\n,4만원 이상,https://www.mangoplate.com//restaurants/YyRGx0...,\n 가로수길에 있는 오랜 참치맛집 e-청우참치위치를 옮기다!!원래...
1,1,청우참치,\n4.5\n,서울특별시 강남구 강남대로 632,\n회 / 스시\n,4만원 이상,https://www.mangoplate.com//restaurants/YyRGx0...,"\n 가성비 너무 좋고, 정말 양질의 참치를 양껏 즐길 수 있어서 ..."
2,2,청우참치,\n4.5\n,서울특별시 강남구 강남대로 632,\n회 / 스시\n,4만원 이상,https://www.mangoplate.com//restaurants/YyRGx0...,\n 가성비 좋은 한국식 참치횟집. 선도가 꽤 좋다. 카운터석도 있...
3,3,청우참치,\n4.5\n,서울특별시 강남구 강남대로 632,\n회 / 스시\n,4만원 이상,https://www.mangoplate.com//restaurants/YyRGx0...,\n 그저 박수만. 참치땡길때 여기만한 곳이 없다.\n
4,4,청우참치,\n4.5\n,서울특별시 강남구 강남대로 632,\n회 / 스시\n,4만원 이상,https://www.mangoplate.com//restaurants/YyRGx0...,\n 맛있다.. 느끼해서 많이 못먹는게 함정.\n
...,...,...,...,...,...,...,...,...
44415,44415,프로젝트얼스,\n 4.5\n ...,서울특별시 성북구 성북로15길 21,\n카페 / 디저트\n,만원 미만,https://www.mangoplate.com//restaurants/2r63mF...,\n 성북동에서 까페를 찾다가 가게되었어요. 구조가 뭔가 특이하고 ...
44416,44416,프로젝트얼스,\n 4.5\n ...,서울특별시 성북구 성북로15길 21,\n카페 / 디저트\n,만원 미만,https://www.mangoplate.com//restaurants/2r63mF...,\n * 한줄평 : 아지트 삼고싶은 성북동 골목 카페성북동 언덕 아...
44417,44417,프로젝트얼스,\n 4.5\n ...,서울특별시 성북구 성북로15길 21,\n카페 / 디저트\n,만원 미만,https://www.mangoplate.com//restaurants/2r63mF...,\n 조용한 성북동에있는 가격도 저렴하고 부니기 좋은 카페 ㅎㅎ 사...
44418,44418,프로젝트얼스,\n 4.5\n ...,서울특별시 성북구 성북로15길 21,\n카페 / 디저트\n,만원 미만,https://www.mangoplate.com//restaurants/2r63mF...,\n 리뷰 썼다가 사람 많아질까 두렵지만... 그렇지만 또 장사가 ...


In [10]:
df=df.drop(["level_0"], axis=1)
df

,이름,평점,주소,주력 메뉴,가격대,망고플레이트 주소,리뷰내용
0,청우참치,\n4.5\n,서울특별시 강남구 강남대로 632,\n회 / 스시\n,4만원 이상,https://www.mangoplate.com//restaurants/YyRGx0...,\n 가로수길에 있는 오랜 참치맛집 e-청우참치위치를 옮기다!!원래...
1,청우참치,\n4.5\n,서울특별시 강남구 강남대로 632,\n회 / 스시\n,4만원 이상,https://www.mangoplate.com//restaurants/YyRGx0...,"\n 가성비 너무 좋고, 정말 양질의 참치를 양껏 즐길 수 있어서 ..."
2,청우참치,\n4.5\n,서울특별시 강남구 강남대로 632,\n회 / 스시\n,4만원 이상,https://www.mangoplate.com//restaurants/YyRGx0...,\n 가성비 좋은 한국식 참치횟집. 선도가 꽤 좋다. 카운터석도 있...
3,청우참치,\n4.5\n,서울특별시 강남구 강남대로 632,\n회 / 스시\n,4만원 이상,https://www.mangoplate.com//restaurants/YyRGx0...,\n 그저 박수만. 참치땡길때 여기만한 곳이 없다.\n
4,청우참치,\n4.5\n,서울특별시 강남구 강남대로 632,\n회 / 스시\n,4만원 이상,https://www.mangoplate.com//restaurants/YyRGx0...,\n 맛있다.. 느끼해서 많이 못먹는게 함정.\n
...,...,...,...,...,...,...,...
44415,프로젝트얼스,\n 4.5\n ...,서울특별시 성북구 성북로15길 21,\n카페 / 디저트\n,만원 미만,https://www.mangoplate.com//restaurants/2r63mF...,\n 성북동에서 까페를 찾다가 가게되었어요. 구조가 뭔가 특이하고 ...
44416,프로젝트얼스,\n 4.5\n ...,서울특별시 성북구 성북로15길 21,\n카페 / 디저트\n,만원 미만,https://www.mangoplate.com//restaurants/2r63mF...,\n * 한줄평 : 아지트 삼고싶은 성북동 골목 카페성북동 언덕 아...
44417,프로젝트얼스,\n 4.5\n ...,서울특별시 성북구 성북로15길 21,\n카페 / 디저트\n,만원 미만,https://www.mangoplate.com//restaurants/2r63mF...,\n 조용한 성북동에있는 가격도 저렴하고 부니기 좋은 카페 ㅎㅎ 사...
44418,프로젝트얼스,\n 4.5\n ...,서울특별시 성북구 성북로15길 21,\n카페 / 디저트\n,만원 미만,https://www.mangoplate.com//restaurants/2r63mF...,\n 리뷰 썼다가 사람 많아질까 두렵지만... 그렇지만 또 장사가 ...


In [11]:
df['평점'] = df['평점'].str.replace('\n','')
df['리뷰내용'] = df['리뷰내용'].str.replace('\n','')
df

,이름,평점,주소,주력 메뉴,가격대,망고플레이트 주소,리뷰내용
0,청우참치,4.5,서울특별시 강남구 강남대로 632,\n회 / 스시\n,4만원 이상,https://www.mangoplate.com//restaurants/YyRGx0...,가로수길에 있는 오랜 참치맛집 e-청우참치위치를 옮기다!!원래 있...
1,청우참치,4.5,서울특별시 강남구 강남대로 632,\n회 / 스시\n,4만원 이상,https://www.mangoplate.com//restaurants/YyRGx0...,"가성비 너무 좋고, 정말 양질의 참치를 양껏 즐길 수 있어서 더 ..."
2,청우참치,4.5,서울특별시 강남구 강남대로 632,\n회 / 스시\n,4만원 이상,https://www.mangoplate.com//restaurants/YyRGx0...,가성비 좋은 한국식 참치횟집. 선도가 꽤 좋다. 카운터석도 있고 ...
3,청우참치,4.5,서울특별시 강남구 강남대로 632,\n회 / 스시\n,4만원 이상,https://www.mangoplate.com//restaurants/YyRGx0...,그저 박수만. 참치땡길때 여기만한 곳이 없다.
4,청우참치,4.5,서울특별시 강남구 강남대로 632,\n회 / 스시\n,4만원 이상,https://www.mangoplate.com//restaurants/YyRGx0...,맛있다.. 느끼해서 많이 못먹는게 함정.
...,...,...,...,...,...,...,...
44415,프로젝트얼스,4.5 ...,서울특별시 성북구 성북로15길 21,\n카페 / 디저트\n,만원 미만,https://www.mangoplate.com//restaurants/2r63mF...,성북동에서 까페를 찾다가 가게되었어요. 구조가 뭔가 특이하고 분위...
44416,프로젝트얼스,4.5 ...,서울특별시 성북구 성북로15길 21,\n카페 / 디저트\n,만원 미만,https://www.mangoplate.com//restaurants/2r63mF...,* 한줄평 : 아지트 삼고싶은 성북동 골목 카페성북동 언덕 아래 ...
44417,프로젝트얼스,4.5 ...,서울특별시 성북구 성북로15길 21,\n카페 / 디저트\n,만원 미만,https://www.mangoplate.com//restaurants/2r63mF...,조용한 성북동에있는 가격도 저렴하고 부니기 좋은 카페 ㅎㅎ 사람들...
44418,프로젝트얼스,4.5 ...,서울특별시 성북구 성북로15길 21,\n카페 / 디저트\n,만원 미만,https://www.mangoplate.com//restaurants/2r63mF...,리뷰 썼다가 사람 많아질까 두렵지만... 그렇지만 또 장사가 잘 ...


In [15]:
gu=[]
for i in range(len(df["주소"])):
    gu.append(df["주소"][i][6:9])
#name_coordinate3=pd.concat([name_coordinate3, pd.Series(gu)], axis=1)
pd.Series(gu).unique()

array(['강남구', '광진구', '구로구', '노원구', '도봉구', '동대문', '동작구', '마포구', '서대문',
       '서초구', '성동구', '송파구', '양천구', '영등포', '용산구', '은평구', '종로구', '중구 ',
       '중랑구', '강북구', '한천로', '강서구', '관악구', '강동구', '금천구', '성북구'],
      dtype=object)

In [21]:
df["구"]=pd.Series(gu)
df

,이름,평점,주소,주력 메뉴,가격대,망고플레이트 주소,리뷰내용,구
0,청우참치,4.5,서울특별시 강남구 강남대로 632,\n회 / 스시\n,4만원 이상,https://www.mangoplate.com//restaurants/YyRGx0...,가로수길에 있는 오랜 참치맛집 e-청우참치위치를 옮기다!!원래 있...,강남구
1,청우참치,4.5,서울특별시 강남구 강남대로 632,\n회 / 스시\n,4만원 이상,https://www.mangoplate.com//restaurants/YyRGx0...,"가성비 너무 좋고, 정말 양질의 참치를 양껏 즐길 수 있어서 더 ...",강남구
2,청우참치,4.5,서울특별시 강남구 강남대로 632,\n회 / 스시\n,4만원 이상,https://www.mangoplate.com//restaurants/YyRGx0...,가성비 좋은 한국식 참치횟집. 선도가 꽤 좋다. 카운터석도 있고 ...,강남구
3,청우참치,4.5,서울특별시 강남구 강남대로 632,\n회 / 스시\n,4만원 이상,https://www.mangoplate.com//restaurants/YyRGx0...,그저 박수만. 참치땡길때 여기만한 곳이 없다.,강남구
4,청우참치,4.5,서울특별시 강남구 강남대로 632,\n회 / 스시\n,4만원 이상,https://www.mangoplate.com//restaurants/YyRGx0...,맛있다.. 느끼해서 많이 못먹는게 함정.,강남구
...,...,...,...,...,...,...,...,...
44415,프로젝트얼스,4.5 ...,서울특별시 성북구 성북로15길 21,\n카페 / 디저트\n,만원 미만,https://www.mangoplate.com//restaurants/2r63mF...,성북동에서 까페를 찾다가 가게되었어요. 구조가 뭔가 특이하고 분위...,성북구
44416,프로젝트얼스,4.5 ...,서울특별시 성북구 성북로15길 21,\n카페 / 디저트\n,만원 미만,https://www.mangoplate.com//restaurants/2r63mF...,* 한줄평 : 아지트 삼고싶은 성북동 골목 카페성북동 언덕 아래 ...,성북구
44417,프로젝트얼스,4.5 ...,서울특별시 성북구 성북로15길 21,\n카페 / 디저트\n,만원 미만,https://www.mangoplate.com//restaurants/2r63mF...,조용한 성북동에있는 가격도 저렴하고 부니기 좋은 카페 ㅎㅎ 사람들...,성북구
44418,프로젝트얼스,4.5 ...,서울특별시 성북구 성북로15길 21,\n카페 / 디저트\n,만원 미만,https://www.mangoplate.com//restaurants/2r63mF...,리뷰 썼다가 사람 많아질까 두렵지만... 그렇지만 또 장사가 잘 ...,성북구


In [30]:
df['구'] = df['구'].str.replace('한천로','강북구')
df["구"].unique()

array(['강남구', '광진구', '구로구', '노원구', '도봉구', '동대문', '동작구', '마포구', '서대문',
       '서초구', '성동구', '송파구', '양천구', '영등포', '용산구', '은평구', '종로구', '중구 ',
       '중랑구', '강북구', '강서구', '관악구', '강동구', '금천구', '성북구'], dtype=object)

In [32]:
df['주력 메뉴'] = df['주력 메뉴'].str.replace('\n','')
df

,이름,평점,주소,주력 메뉴,가격대,망고플레이트 주소,리뷰내용,구
0,청우참치,4.5,서울특별시 강남구 강남대로 632,회 / 스시,4만원 이상,https://www.mangoplate.com//restaurants/YyRGx0...,가로수길에 있는 오랜 참치맛집 e-청우참치위치를 옮기다!!원래 있...,강남구
1,청우참치,4.5,서울특별시 강남구 강남대로 632,회 / 스시,4만원 이상,https://www.mangoplate.com//restaurants/YyRGx0...,"가성비 너무 좋고, 정말 양질의 참치를 양껏 즐길 수 있어서 더 ...",강남구
2,청우참치,4.5,서울특별시 강남구 강남대로 632,회 / 스시,4만원 이상,https://www.mangoplate.com//restaurants/YyRGx0...,가성비 좋은 한국식 참치횟집. 선도가 꽤 좋다. 카운터석도 있고 ...,강남구
3,청우참치,4.5,서울특별시 강남구 강남대로 632,회 / 스시,4만원 이상,https://www.mangoplate.com//restaurants/YyRGx0...,그저 박수만. 참치땡길때 여기만한 곳이 없다.,강남구
4,청우참치,4.5,서울특별시 강남구 강남대로 632,회 / 스시,4만원 이상,https://www.mangoplate.com//restaurants/YyRGx0...,맛있다.. 느끼해서 많이 못먹는게 함정.,강남구
...,...,...,...,...,...,...,...,...
44415,프로젝트얼스,4.5 ...,서울특별시 성북구 성북로15길 21,카페 / 디저트,만원 미만,https://www.mangoplate.com//restaurants/2r63mF...,성북동에서 까페를 찾다가 가게되었어요. 구조가 뭔가 특이하고 분위...,성북구
44416,프로젝트얼스,4.5 ...,서울특별시 성북구 성북로15길 21,카페 / 디저트,만원 미만,https://www.mangoplate.com//restaurants/2r63mF...,* 한줄평 : 아지트 삼고싶은 성북동 골목 카페성북동 언덕 아래 ...,성북구
44417,프로젝트얼스,4.5 ...,서울특별시 성북구 성북로15길 21,카페 / 디저트,만원 미만,https://www.mangoplate.com//restaurants/2r63mF...,조용한 성북동에있는 가격도 저렴하고 부니기 좋은 카페 ㅎㅎ 사람들...,성북구
44418,프로젝트얼스,4.5 ...,서울특별시 성북구 성북로15길 21,카페 / 디저트,만원 미만,https://www.mangoplate.com//restaurants/2r63mF...,리뷰 썼다가 사람 많아질까 두렵지만... 그렇지만 또 장사가 잘 ...,성북구


In [33]:
df.to_csv("data/리뷰_6개구 수정_0112.csv", sep=",")

In [35]:
df=pd.read_csv("data/리뷰_6개구 수정_0112.csv", sep=",", index_col=0)
df

,이름,평점,주소,주력 메뉴,가격대,망고플레이트 주소,리뷰내용,구
0,청우참치,4.5,서울특별시 강남구 강남대로 632,회 / 스시,4만원 이상,https://www.mangoplate.com//restaurants/YyRGx0...,가로수길에 있는 오랜 참치맛집 e-청우참치위치를 옮기다!!원래 있...,강남구
1,청우참치,4.5,서울특별시 강남구 강남대로 632,회 / 스시,4만원 이상,https://www.mangoplate.com//restaurants/YyRGx0...,"가성비 너무 좋고, 정말 양질의 참치를 양껏 즐길 수 있어서 더 ...",강남구
2,청우참치,4.5,서울특별시 강남구 강남대로 632,회 / 스시,4만원 이상,https://www.mangoplate.com//restaurants/YyRGx0...,가성비 좋은 한국식 참치횟집. 선도가 꽤 좋다. 카운터석도 있고 ...,강남구
3,청우참치,4.5,서울특별시 강남구 강남대로 632,회 / 스시,4만원 이상,https://www.mangoplate.com//restaurants/YyRGx0...,그저 박수만. 참치땡길때 여기만한 곳이 없다.,강남구
4,청우참치,4.5,서울특별시 강남구 강남대로 632,회 / 스시,4만원 이상,https://www.mangoplate.com//restaurants/YyRGx0...,맛있다.. 느끼해서 많이 못먹는게 함정.,강남구
...,...,...,...,...,...,...,...,...
44415,프로젝트얼스,4.5 ...,서울특별시 성북구 성북로15길 21,카페 / 디저트,만원 미만,https://www.mangoplate.com//restaurants/2r63mF...,성북동에서 까페를 찾다가 가게되었어요. 구조가 뭔가 특이하고 분위...,성북구
44416,프로젝트얼스,4.5 ...,서울특별시 성북구 성북로15길 21,카페 / 디저트,만원 미만,https://www.mangoplate.com//restaurants/2r63mF...,* 한줄평 : 아지트 삼고싶은 성북동 골목 카페성북동 언덕 아래 ...,성북구
44417,프로젝트얼스,4.5 ...,서울특별시 성북구 성북로15길 21,카페 / 디저트,만원 미만,https://www.mangoplate.com//restaurants/2r63mF...,조용한 성북동에있는 가격도 저렴하고 부니기 좋은 카페 ㅎㅎ 사람들...,성북구
44418,프로젝트얼스,4.5 ...,서울특별시 성북구 성북로15길 21,카페 / 디저트,만원 미만,https://www.mangoplate.com//restaurants/2r63mF...,리뷰 썼다가 사람 많아질까 두렵지만... 그렇지만 또 장사가 잘 ...,성북구


In [38]:
from sklearn.cluster import DBSCAN
from geopy.distance import great_circle
from shapely.geometry import MultiPoint

In [4]:
df = pd.read_csv("data/맛집리스트_위도경도분리(0108).csv", sep=",", index_col=0)
df

,이름,평점,주소,좌표,구,좌표2,위도,경도
0,청우참치,4.5,서울특별시 강남구 강남대로 632,"(37.5182046, 127.019356)",강남구,"37.5182046, 127.019356",37.518205,127.019356
1,정돈,4.3,서울특별시 강남구 강남대로110길 26 1F,"(37.5035911, 127.0269451)",강남구,"37.5035911, 127.0269451",37.503591,127.026945
2,꼼다비뛰드,4.5,서울특별시 강남구 강남대로110길 62,"(37.5033264, 127.0294865)",강남구,"37.5033264, 127.0294865",37.503326,127.029487
3,노들강,4.4,서울특별시 강남구 강남대로114길 19,"(37.5061534, 127.0258181)",강남구,"37.5061534, 127.0258181",37.506153,127.025818
4,영동소금구이,4.4,서울특별시 강남구 강남대로124길 10 1F,"(37.5088173, 127.0234738)",강남구,"37.5088173, 127.0234738",37.508817,127.023474
...,...,...,...,...,...,...,...,...
4649,막회무침나라횟집,-,서울특별시 금천구 시흥대로71길 40,"(37.45521069999999, 126.8993725)",금천구,"37.45521069999999, 126.8993725",37.455211,126.899372
4650,포차인닭갈비,-,서울특별시 금천구 가산디지털1로 168 우림라이온스밸리 A동 지하,"(37.4800992, 126.8826202)",금천구,"37.4800992, 126.8826202",37.480099,126.882620
4651,명화곱창,-,서울특별시 금천구 벚꽃로38길 5,"(37.4818785, 126.8832991)",금천구,"37.4818785, 126.8832991",37.481879,126.883299
4652,혜화동돈까스,-,서울특별시 금천구 금하로5길 4 1F,"(37.4543357, 126.8988835)",금천구,"37.4543357, 126.8988835",37.454336,126.898883


In [18]:
df["좌표2"]=list(df["좌표2"])
len(df["좌표"].unique())
len(df["이름"].unique())

25

In [68]:
df2=df.drop_duplicates(["좌표"])
df2.to_csv("data/좌표 중복값 제거 리스트.csv", sep=",", index=False)

In [41]:
coords = df2[["위도", "경도"]].to_numpy()

In [51]:
clusters=[]
kms_per_radian = 6371.0088
epsilon = 0.2 / kms_per_radian
db = DBSCAN(eps=epsilon, min_samples=10, algorithm='ball_tree', metric='haversine').fit(np.radians(coords))
print(db)
print("-"*80)
cluster_labels = db.labels_
print(cluster_labels)
print("-"*80)
num_clusters = len(set(cluster_labels))
print(set(cluster_labels))
print("-"*80)
clusters = pd.Series([coords[cluster_labels == n] for n in range(num_clusters)])
print(clusters)
print("-"*80)
print('Number of clusters: {}'.format(num_clusters))

DBSCAN(algorithm='ball_tree', eps=3.139220275445233e-05, metric='haversine',
       min_samples=10)
--------------------------------------------------------------------------------
[-1 -1 -1 ... 67 67 -1]
--------------------------------------------------------------------------------
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, -1}
--------------------------------------------------------------------------------
0     [[37.5186497, 127.0242496], [37.5190769, 127.0...
1     [[37.5215969, 127.0357429], [37.5225343, 127.0...
2     [[37.5453934, 127.0718498], [37.5443865, 127.0...
3     [[37.5481363, 127.0715056], [37.5475035, 127.0...
4     [[37.5036144, 126.8900917], [37.5037511, 126.8...
                            ...                        
65    [[37.4806236, 126.95

In [52]:
for i in range(len(clusters)):
    print(len(clusters[i]))

18
48
90
8
16
27
37
36
14
46
12
14
62
24
10
28
12
34
13
25
13
11
79
10
29
99
19
10
14
12
52
38
10
13
51
37
20
19
14
17
33
12
18
11
44
12
42
10
32
13
10
20
11
13
41
14
11
11
68
40
20
59
28
23
15
117
12
24
21
0


In [58]:
centroid=[]

for i in range(len(clusters)-1):
    lan=[]
    lon=[]
    for j in range(len(clusters[i])):
        lan.append(clusters[i][j][0])
        lon.append(clusters[i][j][1])
    centroid.append((sum(lan)/len(clusters[i]), sum(lon)/len(clusters[i])))

In [56]:
clusters[65]

array([[ 37.4806236, 126.9573674],
       [ 37.479212 , 126.9572004],
       [ 37.4779266, 126.9568418],
       [ 37.4787665, 126.9576725],
       [ 37.477823 , 126.9550576],
       [ 37.4776242, 126.9582039],
       [ 37.4778012, 126.9588719],
       [ 37.4782232, 126.9553261],
       [ 37.4785401, 126.9569316],
       [ 37.4782015, 126.956336 ],
       [ 37.4783172, 126.9560743],
       [ 37.4771776, 126.9583514],
       [ 37.4774033, 126.9584998],
       [ 37.4787237, 126.9534468],
       [ 37.4821057, 126.9536879],
       [ 37.4790093, 126.9528334],
       [ 37.4797599, 126.9588447],
       [ 37.4788509, 126.9541697],
       [ 37.4790279, 126.9535036],
       [ 37.4788977, 126.9526427],
       [ 37.4820616, 126.954099 ],
       [ 37.4804505, 126.9571527],
       [ 37.4798639, 126.9551087],
       [ 37.4763171, 126.9587523],
       [ 37.4798571, 126.9555667],
       [ 37.4789727, 126.9542856],
       [ 37.4801549, 126.9529757],
       [ 37.4776206, 126.957287 ],
       [ 37.4795649,

In [57]:
#for i in range(len(clusters)-1):
lan=[]
lon=[]
for j in range(len(clusters[65])):
    lan.append(clusters[65][j][0])
    lon.append(clusters[65][j][1])
    
for i in range(len(lan)):
    print(haversine((lan[0],lon[0]), (lan[i],lon[i])))

0.0
0.15765319896532096
0.3034583468224105
0.2082479765256709
0.37218372901654856
0.34158912811908737
0.34076196566158584
0.3220063539912398
0.234844854669632
0.28428767123144444
0.2806987619260114
0.39289314617357857
0.37176252625472095
0.40536068864894303
0.3641069766634666
0.4385067685850322
0.16191543927165075
0.3441995144169245
0.38435169475549713
0.4589612924841435
0.3297610471596556
0.0270073717553277
0.2164713726808922
0.49420944633812003
0.18031013674448546
0.32810117605813593
0.3910131196822819
0.3339941856862861
0.41224518421731554
0.4109138062743715
0.3558196860155561
0.2697608348990584
0.31801714555837024
0.18610571764025996
0.291255817167189
0.5346612215773772
0.63271577288264
0.2982768684373137
0.20754316468107178
0.4699931670345007
0.2353463691033767
0.37107452256947365
0.41366635310379307
0.45510630086823983
0.3962401393390927
0.48905770253031894
0.4689025950337671
0.7290740397904585
0.30354882289808033
0.28108779440060105
0.04195452793557115
0.4704183096238765
0.23079

In [144]:
centroid[0][0]

37.5212982119403

In [60]:
map=folium.Map(location=[centroid[0][0], centroid[0][1]], zoom_start=12)
#folium.CircleMarker([center_point[0], center_point[1]], radius=100, color="skyblue", fill_color="skyblue").add_to(map)
for i in range(len(centroid)):
    folium.Marker([centroid[i][0], centroid[i][1]], popup=i).add_to(map)
    #folium.PolyLine(locations=[coordinate[i], center_point],weight=2, color = 'blue').add_to(map)/
map

In [101]:
# db = DBSCAN(eps=1.5/kms_per_radian, min_samples=5, algorithm='ball_tree', metric='haversine').fit(np.radians(coords))
# cluster_labels = db.labels_
# num_clusters = len(set(cluster_labels))
# clusters = pd.Series([coords[cluster_labels == n] for n in range(num_clusters)])
# print('Number of clusters: {}'.format(num_clusters))

Number of clusters: 2


In [67]:

map=folium.Map(location=[centroid[0][0], centroid[0][1]], zoom_start=12)
#folium.CircleMarker([center_point[0], center_point[1]], radius=100, color="skyblue", fill_color="skyblue").add_to(map)
for i in range(len(centroid)):
    folium.Marker([centroid[i][0], centroid[i][1]], icon=folium.Icon(color="red")).add_to(map)
    folium.Circle([centroid[i][0], centroid[i][1]], radius=500, color="skyblue", fill_color="skyblue").add_to(map)
    #folium.PolyLine(locations=[coordinate[i], center_point],weight=2, color = 'blue').add_to(map)/

#for i in range(len(coords)):
for i in range(3870, 4057):
     folium.Marker([coords[i][0], coords[i][1]], color="red").add_to(map)
map


In [66]:
df3=df2.reset_index()
df3[df3["구"]=="관악구"]

,index,이름,평점,주소,좌표,구,좌표2,위도,경도
3870,4273,따빠디또,4.8,서울특별시 관악구 행운1길 40 상가 104호,"(37.4806236, 126.9573674)",관악구,"37.4806236, 126.9573674",37.480624,126.957367
3871,4274,빠사삭,4.6,서울특별시 관악구 신림로17길 41,"(37.4688176, 126.9364921)",관악구,"37.4688176, 126.9364921",37.468818,126.936492
3872,4275,루비정,4.6,서울특별시 관악구 남부순환로 1690,"(37.4836767, 126.9379912)",관악구,"37.4836767, 126.9379912",37.483677,126.937991
3873,4276,오목눈이,4.5,서울특별시 관악구 장군봉1길 32 2F,"(37.4816193, 126.9417673)",관악구,"37.4816193, 126.9417673",37.481619,126.941767
3874,4277,뉴욕택시디저트,4.5,서울특별시 관악구 남부순환로 1870 1F,"(37.479212, 126.9572004)",관악구,"37.479212, 126.9572004",37.479212,126.957200
...,...,...,...,...,...,...,...,...,...
4052,4464,길초밥,3.6,서울특별시 관악구 남부순환로 1600-24 1F,"(37.4834123, 126.9291608)",관악구,"37.4834123, 126.9291608",37.483412,126.929161
4053,4465,생각보다맛있는집,3.6,서울특별시 관악구 봉천로 236,"(37.48913, 126.9275346)",관악구,"37.48913, 126.9275346",37.489130,126.927535
4054,4466,노다지,3.6,서울특별시 관악구 관악로16길 36 2F,"(37.4797237, 126.9545454)",관악구,"37.4797237, 126.9545454",37.479724,126.954545
4055,4467,황토방,3.6,서울특별시 관악구 관악로12길 6 2F,"(37.4780533, 126.9527314)",관악구,"37.4780533, 126.9527314",37.478053,126.952731


DBSCAN(eps=1, metric='precomputed', min_samples=2)